In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!unzip /content/drive/MyDrive/MIMIC/mimic_cxr-IV.zip -d /content/train_images_folder

Streaming output truncated to the last 5000 lines.
  inflating: /content/train_images_folder/re_512_3ch/Train/s59521988.jpg  
  inflating: /content/train_images_folder/re_512_3ch/Train/s59521996.jpg  
  inflating: /content/train_images_folder/re_512_3ch/Train/s59522022.jpg  
  inflating: /content/train_images_folder/re_512_3ch/Train/s59522128.jpg  
  inflating: /content/train_images_folder/re_512_3ch/Train/s59522228.jpg  
  inflating: /content/train_images_folder/re_512_3ch/Train/s59522366.jpg  
  inflating: /content/train_images_folder/re_512_3ch/Train/s59522423.jpg  
  inflating: /content/train_images_folder/re_512_3ch/Train/s59522427.jpg  
  inflating: /content/train_images_folder/re_512_3ch/Train/s59522605.jpg  
  inflating: /content/train_images_folder/re_512_3ch/Train/s59522677.jpg  
  inflating: /content/train_images_folder/re_512_3ch/Train/s59522723.jpg  
  inflating: /content/train_images_folder/re_512_3ch/Train/s59522888.jpg  
  inflating: /content/train_images_folder/re_512_

In [4]:
#!pip install monai

In [5]:
import torch
import cv2
import torchvision.transforms as transforms
import tensorflow as tf
import numpy as np
#from monai.networks.nets import DenseNet121
import os
import json
from PIL import Image
from tqdm import tqdm
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset, WeightedRandomSampler
import torch.nn.functional as F
from sklearn.metrics import f1_score
import random
import statistics

In [6]:
train_json_path = "/content/drive/MyDrive/MIMIC/train_data.json"
unique_labels_path = "/content/drive/MyDrive/MIMIC/unique_labels.json"
images_folder = "/content/train_images_folder/re_512_3ch/Train"
output_folder = "/content/Train_dataset"

os.makedirs(output_folder, exist_ok=True)

with open(train_json_path, "r") as f:
    train_data = json.load(f)

with open(unique_labels_path, "r") as f:
    unique_labels = json.load(f)

label_to_index = {label: i for i, label in enumerate(unique_labels)}

dataset = []
for entry in tqdm(train_data, desc="Processing JSON entries"):
    image_id = entry["id"]
    labels = entry["label"].strip("'").split("', '") if entry["label"] else []

    if not labels:
        continue

    label_vector = np.zeros(len(unique_labels), dtype=np.float32)
    for label in labels:
        if label in label_to_index:
          label_vector[label_to_index[label]] = 1

    image_path = os.path.join(images_folder, f"{image_id}.jpg")
    if os.path.exists(image_path):
        with Image.open(image_path) as img:

            img_gray = img.convert("L")

            output_image_path = os.path.join(output_folder, f"{image_id}.jpg")
            img_gray.save(output_image_path)

            dataset.append({"image_path": output_image_path, "label_vector": label_vector.tolist(), "counter": 0})

dataset_json_path = os.path.join(output_folder, "dataset.json")
with open(dataset_json_path, "w") as f:
    json.dump(dataset, f)

print(f"Processed dataset saved at {output_folder}")

Processing JSON entries: 100%|██████████| 89395/89395 [04:33<00:00, 326.37it/s]


Processed dataset saved at /content/Train_dataset


In [7]:
class CustomDataset(Dataset):
  def __init__(self, data_json, transform=None, sample_size=9000, seed=42):
        if sample_size and len(data_json) > sample_size:
            random.seed(seed)  # برای reproducibility
            self.data = random.sample(data_json, sample_size)
        else:
            self.data = data_json

        self.transform = transform

  def __len__(self):
        return len(self.data)

  def __getitem__(self, idx):
        img_path = self.data[idx]['image_path']
        image = Image.open(img_path).convert('L')
        if self.transform:
            image = self.transform(image)
        label = torch.tensor(self.data[idx]['label_vector'], dtype=torch.float16)
        """
        old = new
        float32 = float16
        """
        #counter = torch.tensor(self.data[idx]['counter'], dtype=torch.float32)

        return image, label

In [8]:
with open('/content/Train_dataset/dataset.json', 'r') as f:
    dataset_json = json.load(f)

In [9]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.5], [0.5])
])

In [10]:
dataset = CustomDataset(dataset_json, transform=transform)

In [11]:
import torch
import torch.nn as nn
from torch.distributions import Categorical
import torch.nn.functional as F

"""
class Agent(nn.Module):
    def __init__(self, input_dim, num_classes, d_model=728, nhead=8, num_layers=10, dim_feedforward=512, dropout=0.5):
        super(Agent, self).__init__()
        self.embedding = nn.Linear(input_dim, d_model)  # Project input to d_model
        encoder_layer = nn.TransformerEncoderLayer(d_model=d_model, nhead=nhead, dim_feedforward=dim_feedforward, dropout=dropout, batch_first=True)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)
        self.classifier = nn.Linear(d_model, num_classes)  # Final classifier layer

    def forward(self, x):
        # Add positional encoding to input embeddings
        x = self.embedding(x)
        x = self.transformer_encoder(x.unsqueeze(1))  # Add sequence dimension
        x = x.mean(dim=1)  # Aggregate over sequence dimension
        return self.classifier(x)

"""

'\nclass Agent(nn.Module):\n    def __init__(self, input_dim, num_classes, d_model=728, nhead=8, num_layers=10, dim_feedforward=512, dropout=0.5):\n        super(Agent, self).__init__()\n        self.embedding = nn.Linear(input_dim, d_model)  # Project input to d_model\n        encoder_layer = nn.TransformerEncoderLayer(d_model=d_model, nhead=nhead, dim_feedforward=dim_feedforward, dropout=dropout, batch_first=True)\n        self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)\n        self.classifier = nn.Linear(d_model, num_classes)  # Final classifier layer\n\n    def forward(self, x):\n        # Add positional encoding to input embeddings\n        x = self.embedding(x)\n        x = self.transformer_encoder(x.unsqueeze(1))  # Add sequence dimension\n        x = x.mean(dim=1)  # Aggregate over sequence dimension\n        return self.classifier(x)\n\n'

In [12]:
import torch
import torch.nn as nn
from timm.models import create_model
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import torch.optim as optim

class TinyViTClassifier(nn.Module):
    def __init__(self, num_classes=14):
        super(TinyViTClassifier, self).__init__()
        # Load a pretrained Tiny ViT model from timm
        self.vit = create_model(
            'vit_tiny_patch16_224',  # Tiny ViT with patch size 16
            pretrained=True,
            num_classes=num_classes
        )

        # Add a channel expansion layer for grayscale inputs
        self.expand_channels = nn.Conv2d(1, 3, kernel_size=1)

    def forward(self, x):
        # Expand grayscale input to 3 channels
        x = self.expand_channels(x)
        #x = nn.functional.interpolate(x, size=(224, 224), mode='bilinear', align_corners=False)
        # Forward pass through the Vision Transformer
        x = self.vit(x)
        return x

In [13]:
class TinyViTevaluation(nn.Module):
    def __init__(self, num_classes=3):
        super(TinyViTevaluation, self).__init__()
        # Load a pretrained Tiny ViT model from timm
        self.vit = create_model(
            'vit_tiny_patch16_224',  # Tiny ViT with patch size 16
            pretrained=True,
            num_classes=num_classes
        )

        # Add a channel expansion layer for grayscale inputs
        self.expand_channels = nn.Conv2d(1, 3, kernel_size=1)

    def forward(self, x):
        # Expand grayscale input to 3 channels
        x = self.expand_channels(x)
        #x = nn.functional.interpolate(x, size=(224, 224), mode='bilinear', align_corners=False)
        # Forward pass through the Vision Transformer
        x = self.vit(x)
        return x

In [14]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [15]:
"""
embedding_model = DenseNet121(spatial_dims=2, in_channels=1, out_channels=512, pretrained=True).to(device)
"""

'\nembedding_model = DenseNet121(spatial_dims=2, in_channels=1, out_channels=512, pretrained=True).to(device)\n'

In [16]:
model_val = TinyViTevaluation(num_classes=3).to(device) ########### change
model_class = TinyViTClassifier(num_classes=14).to(device)

val_optimizer = torch.optim.Adam(model_val.parameters(), lr=0.0001) #weight_decay = 1e-2
model_optimizer = torch.optim.Adam(model_class.parameters(), lr=0.0001)

criterion = nn.BCEWithLogitsLoss()
criterion_E = nn.CrossEntropyLoss()  # Agent(evaluation)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model.safetensors:   0%|          | 0.00/22.9M [00:00<?, ?B/s]

In [17]:
from torch.utils.data import random_split

train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size

train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

In [18]:
print(len(train_dataset), len(test_dataset))

7200 1800


In [19]:
from torch.utils.data import Subset
import random

# شافل کردن اندیس‌ها
indices = list(range(len(train_dataset)))
random.seed(42)  # برای reproducibility
random.shuffle(indices)

# ساختن دیتاست شافل‌شده
shuffled_dataset = Subset(train_dataset, indices)

In [20]:
print(len(shuffled_dataset), len(test_dataset))

7200 1800


In [ ]:
"""
### NEW ###
"""
import torch.nn.functional as F
from collections import Counter
from sklearn.mixture import GaussianMixture
import numpy as np

epochs = 100
batch_size = 100

# DataLoader بدون shuffle برای حفظ ترتیب تصاویر
train_loader = DataLoader(shuffled_dataset, batch_size=batch_size, shuffle=False)

for epoch in range(epochs):
    print("=" * 80)
    print(f"Epoch {epoch+1}/{epochs}")
    print("-" * 80)

    model_class.train()
    loss_list = []

    total_loss_class = 0
    total_samples = 0

    # ----------- مرحله اول: آموزش مدل ۱۴ کلاسه و استخراج loss ----------- #
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        outputs = model_class(images)
        loss = criterion(outputs, labels)

        model_optimizer.zero_grad()
        loss.backward()
        model_optimizer.step()

        total_loss_class += loss.item() * images.size(0)
        total_samples += images.size(0)

        # ذخیره‌ی loss برای هر تصویر
        with torch.no_grad():
            preds = model_class(images)
            per_sample_loss = F.binary_cross_entropy_with_logits(preds, labels, reduction='none')
            loss_per_image = per_sample_loss.sum(dim=1).cpu().numpy().tolist()
            loss_list.extend(loss_per_image)

    avg_loss_class = total_loss_class / total_samples
    print(f"Average classification loss (14-class model): {avg_loss_class:.4f}")

    # ----------- مرحله دوم: خوشه‌بندی GMM روی lossها ----------- #
    print("\nClustering losses with GMM...")
    loss_array = np.array(loss_list).reshape(-1, 1)
    gmm = GaussianMixture(n_components=3, random_state=42).fit(loss_array)
    cluster_ids = gmm.predict(loss_array)

    means = gmm.means_.flatten()
    sorted_clusters = np.argsort(means)
    cluster_to_difficulty = {sorted_clusters[0]: 0, sorted_clusters[1]: 1, sorted_clusters[2]: 2}
    difficulty_labels = [cluster_to_difficulty[c] for c in cluster_ids]

    # گزارش آماری خوشه‌ها
    counter = Counter(difficulty_labels)
    print("\nCluster distribution:")
    for d in [0, 1, 2]:
        count = counter[d]
        percent = 100 * count / len(difficulty_labels)
        print(f"  Difficulty {d} → {count} samples ({percent:.2f}%)")

    difficulty_labels_tensor = torch.tensor(difficulty_labels, dtype=torch.long).to(device)

    # ----------- مرحله سوم: آموزش مدل ارزش‌گذاری (۳ کلاس) ----------- #
    model_val.train()
    total_loss_val = 0
    idx = 0

    for images, _ in train_loader:
        images = images.to(device)
        batch_labels = difficulty_labels_tensor[idx:idx + images.size(0)]
        idx += images.size(0)

        outputs = model_val(images)
        loss_val = criterion_E(outputs, batch_labels)

        val_optimizer.zero_grad()
        loss_val.backward()
        val_optimizer.step()

        total_loss_val += loss_val.item() * images.size(0)

    avg_loss_val = total_loss_val / len(difficulty_labels)
    print(f"\nAverage Evaluation loss (3-class model): {avg_loss_val:.4f}")
    print("=" * 80 + "\n")


Epoch 1/100
--------------------------------------------------------------------------------
Average classification loss (14-class model): 0.3852

Clustering losses with GMM...

Cluster distribution:
  Difficulty 0 → 4516 samples (62.72%)
  Difficulty 1 → 2069 samples (28.74%)
  Difficulty 2 → 615 samples (8.54%)

Average Evaluation loss (3-class model): 0.8830

Epoch 2/100
--------------------------------------------------------------------------------
Average classification loss (14-class model): 0.3549

Clustering losses with GMM...

Cluster distribution:
  Difficulty 0 → 4008 samples (55.67%)
  Difficulty 1 → 2534 samples (35.19%)
  Difficulty 2 → 658 samples (9.14%)

Average Evaluation loss (3-class model): 0.9117

Epoch 3/100
--------------------------------------------------------------------------------
Average classification loss (14-class model): 0.3398

Clustering losses with GMM...

Cluster distribution:
  Difficulty 0 → 3402 samples (47.25%)
  Difficulty 1 → 2921 samples (

In [ ]:
"""
############################################## OLD ############################################
"""

epoch_num = 10
train_batch = DataLoader(dataset, batch_size = 100, shuffle=True)
from sklearn.mixture import GaussianMixture
import numpy as np
for i in range(epoch_num):
    c = 0
    print("=" * 120)
    print(f"Training epoch {i+1}/{epoch_num} :")
    for images, labels in train_batch:
        c += 1
        images, labels = images.to(device), labels.to(device)
        #embedding_model.eval()
        #images_embedd_vector = embedding_model(images)
        #print(f"images embedd size: {images_embedd_vector.size()}")
        #print(f"labels size: {labels.size()}")
        #print(f"images size: {images.size()}")
        losses_of_batch = []
        print(f"Classifier process : epoch{i+1}, batch {c} th")
        for image, label in zip(images, labels):
            image = image.unsqueeze(0)
            label = label.unsqueeze(0)
            if (i+1) % 4 == 0:
                model.train()
            else:
                model.eval()
            #print(f"image size: {image.size()}")
            #print(f"label size: {label.size()}")
            y_hat = model(image)
            #print(f"y_hat size: {y_hat.size()}")
            loss = criterion(y_hat, label)
            if torch.isnan(loss):
                loss = torch.tensor(1e6, device=loss.device)  # Replace NaN with a large value
                print("Warning: Encountered NaN loss. Replacing with a large value.")
            losses_of_batch.append(loss.item())
            if (i+1) % 4 == 0:
                loss.backward()
                model_optimizer.step()

        print(f"Average of Classifier losses on {len(losses_of_batch)} data :> {statistics.mean(losses_of_batch)}, Min of Classifier losses :> {min(losses_of_batch)}, Max of Classifier losses :> {max(losses_of_batch)}, Median of Classifier losses :> {statistics.median(losses_of_batch)}")

        data = np.array(losses_of_batch).reshape(-1, 1)
        n_components = 3
        gmm = GaussianMixture(n_components=n_components,random_state=42)
        gmm.fit(data)
        clusters = gmm.predict(data)
        print(f"Means of Clusters :> {gmm.means_.flatten()}")
        max_loss_index = list(gmm.means_.flatten()).index(max(gmm.means_.flatten()))
        min_loss_index = list(gmm.means_.flatten()).index(min(gmm.means_.flatten()))

        synthetic_target = []
        for j in range(len(clusters)):
            if clusters[j] == max_loss_index:
                synthetic_target.append(0)
            elif clusters[j] == min_loss_index:
                synthetic_target.append(1)
            else:
                synthetic_target.append(2)

        synthetic_target = torch.tensor(synthetic_target, dtype=torch.float16)
        """
        new = old
        float16 = long
        """
        synthetic_target = synthetic_target.to(device)

        #unique_rows, counts = torch.unique(synthetic_target, dim=0, return_counts=True)
        #result = {tuple(row.tolist()): count.item() for row, count in zip(unique_rows, counts)}
        #result = {tuple(row.tolist()): count.item() if count.dim() == 0 else count.tolist() for row, count in zip(unique_rows, counts)}
        unique, counts = torch.unique(synthetic_target, return_counts=True)
        result = {int(k.item()): v.item() for k, v in zip(unique, counts)}
        print(f"Dummy Target Report :> {result}")
        true_positives = {0: 0, 1: 0, 2: 0}
        agent_model.train()
        out_of_agent = agent_model(images_embedd_vector)
        loss_A = criterion_A(out_of_agent , synthetic_target)
        _, predicted = torch.max(out_of_agent, 1)
        correct = (predicted == synthetic_target).sum().item()
        for label in [0, 1, 2]:
            true_positives[label] = ((predicted == label) & (synthetic_target == label)).sum().item()
        loss_A.backward()
        agent_optimizer.step()
        print("+" + "-" * 60 + "+")
        print(f"|{' ' * 25}{round(loss_A.item(), 5)}{' ' * 28}|\n")
        print(f"|{' ' * 26}{correct}{' ' * 31}|\n")
        print(f"|{' ' * 20}{true_positives}{' ' * 25}|")
        print("+" + "-" * 60 + "+")
        print('\n')
    print("=" * 120)
    print('\n')


Streaming output truncated to the last 5000 lines.
|                         2.27361                            |

|                          22                               |

|                    {0: 0, 1: 22, 2: 0}                         |
+------------------------------------------------------------+


Classifier process : epoch2, batch 413 th
Average of Classifier losses on 100 data :> 0.8663821512460709, Min of Classifier losses :> 0.6287833452224731, Max of Classifier losses :> 1.0412495136260986, Median of Classifier losses :> 0.8749604225158691
Means of Clusters :> [0.77491364 0.87315653 0.95796427]
Dummy Target Report :> {0: 27, 1: 29, 2: 44}
+------------------------------------------------------------+
|                         1.77613                            |

|                          31                               |

|                    {0: 0, 1: 29, 2: 2}                         |
+------------------------------------------------------------+


Classifier proc